> NOTE: This is a reader‑optimized version of the original NVIDIA "An Even Easier Introduction to CUDA" notebook.
>
> Modifications (by Ralph Cajipe):
> - Condensed verbose paragraphs into concise, skimmable explanations.
> - Preserved all original external links and code semantics.
> - Standardized terminology (host/device, kernel, grid/block, grid‑stride loop).
> - Highlighted progressive learning steps (CPU -> single thread -> block -> multi‑block grid).
> - Removed redundancy; grouped related concepts logically.
> - Kept code cells functionally identical (aside from formatting neutrality).
> - Streamlined exercises and resource lists without losing intent.
>
> Goal: Faster comprehension while retaining technical accuracy and reference.


# An Even Easier Introduction to CUDA


This notebook accompanies Mark Harris's blog post [_An Even Easier Introduction to CUDA_](https://developer.nvidia.com/blog/even-easier-introduction-cuda/).

Want more? See the [NVIDIA DLI](https://nvidia.com/dli) courses:
- C/C++: [_Fundamentals of Accelerated Computing with CUDA C/C++_](https://courses.nvidia.com/courses/course-v1:DLI+C-AC-01+V1/about)
- Python: [_Fundamentals of Accelerated Computing with CUDA Python_](https://courses.nvidia.com/courses/course-v1:DLI+C-AC-02+V1/about)

These offer GPU lab access, Nsight Systems profiling, many exercises, and a certificate. For intermediate/advanced material, browse the DLI [_Accelerated Computing_ catalog](https://www.nvidia.com/en-us/training/online/).


<img src="https://developer.download.nvidia.com/training/courses/T-AC-01-V1/CUDA_Cube_1K.jpeg" width="400">

This is a minimal intro to CUDA, NVIDIA's parallel computing platform. CUDA C++ lets you launch thousands of lightweight GPU threads for data-parallel work (core to modern AI & [Deep Learning](https://developer.nvidia.com/deep-learning)).

Prereqs: C/C++ basics plus a CUDA‑capable NVIDIA GPU (any recent GPU or a cloud instance) and the [CUDA Toolkit](https://developer.nvidia.com/cuda-toolkit) installed.

Goal: Start from a plain CPU array add, then progressively parallelize it on the GPU while learning:
1. Kernels (`__global__`) & launches `<<< >>>`
2. Unified Memory (`cudaMallocManaged`)
3. Thread indexing (`threadIdx`, `blockIdx`)
4. Blocks vs grid scaling
5. Grid-stride loops for flexibility

Let's begin.


<img src="https://developer-blogs.nvidia.com/wp-content/uploads/2017/01/cuda_ai_cube-625x625.jpg" width="400">

## Starting Simple

We'll begin with a CPU C++ program that adds two float arrays of 1M elements.


In [1]:
%%writefile add.cpp

#include <iostream>
#include <math.h>

// function to add the elements of two arrays
void add(int n, float *x, float *y)
{
  for (int i = 0; i < n; i++)
      y[i] = x[i] + y[i];
}

int main(void)
{
  int N = 1<<20; // 1M elements

  float *x = new float[N];
  float *y = new float[N];

  // initialize x and y arrays on the host
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Run kernel on 1M elements on the CPU
  add(N, x, y);

  // Check for errors (all values should be 3.0f)
  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = fmax(maxError, fabs(y[i]-3.0f));
  std::cout << "Max error: " << maxError << std::endl;

  // Free memory
  delete [] x;
  delete [] y;

  return 0;
}

Writing add.cpp


Running the previous cell writes `add.cpp`.
Next: compile and then run it.


In [2]:
%%shell
g++ add.cpp -o add

Then run it:

In [3]:
%%shell
./add

Max error: 0


Output shows correct result (max error 0). To move work to the GPU turn `add` into a CUDA kernel by adding `__global__`. This marks it as device code callable from host code.


```cpp
// CUDA Kernel function to add the elements of two arrays on the GPU
__global__
void add(int n, float *x, float *y)
{
  for (int i = 0; i < n; i++)
      y[i] = x[i] + y[i];
}
```

Terminology: `__global__` function = kernel. GPU code = device code. CPU code = host code.


## Memory Allocation in CUDA

Use [Unified Memory](https://developer.nvidia.com/blog/unified-memory-in-cuda-6/) for simplicity: one pointer usable on CPU and GPU. Replace `new/delete` with `cudaMallocManaged` / `cudaFree`.


```cpp
  // Allocate Unified Memory -- accessible from CPU or GPU
  float *x, *y;
  cudaMallocManaged(&x, N*sizeof(float));
  cudaMallocManaged(&y, N*sizeof(float));

  ...

  // Free memory
  cudaFree(x);
  cudaFree(y);
```

Launch kernels with `<<<gridSize, blockSize>>>`. For now `<<<1,1>>>` runs one thread.


```cpp
add<<<1, 1>>>(N, x, y);
```

Kernels launch asynchronously; call `cudaDeviceSynchronize()` before reading results. Full minimal GPU version:


In [4]:
%%writefile add.cu

#include <iostream>
#include <math.h>
// Kernel function to add the elements of two arrays
__global__
void add(int n, float *x, float *y)
{
  for (int i = 0; i < n; i++)
    y[i] = x[i] + y[i];
}

int main(void)
{
  int N = 1<<20
 ;
  float *x, *y;

  // Allocate Unified Memory – accessible from CPU or GPU
  cudaMallocManaged(&x, N*sizeof(float));
  cudaMallocManaged(&y, N*sizeof(float));

  // initialize x and y arrays on the host
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Run kernel on 1M elements on the GPU
  add<<<1, 1>>>(N, x, y);

  // Wait for GPU to finish before accessing on host
  cudaDeviceSynchronize();

  // Check for errors (all values should be 3.0f)
  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = fmax(maxError, fabs(y[i]-3.0f));
  std::cout << "Max error: " << maxError << std::endl;

  // Free memory
  cudaFree(x);
  cudaFree(y);

  return 0;
}

Writing add.cu


In [5]:
%%shell

nvcc add.cu -o add_cuda
./add_cuda

Max error: 1


This kernel is only correct for a single thread. Multiple threads would redundantly traverse the whole array and race on each element.


## Profile it!

Profile runtime with `nvprof ./add_cuda` (CLI profiler from the CUDA Toolkit).


In [6]:
%%shell

nvprof ./add_cuda

==1238== NVPROF is profiling process 1238, command: ./add_cuda
Max error: 1
==1238== Profiling application: ./add_cuda
==1238== Profiling result:
No kernels were profiled.
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
      API calls:   96.30%  216.98ms         2  108.49ms  55.053us  216.93ms  cudaMallocManaged
                    3.50%  7.8856ms         1  7.8856ms  7.8856ms  7.8856ms  cudaLaunchKernel
                    0.13%  292.94us         2  146.47us  117.54us  175.40us  cudaFree
                    0.06%  126.09us       114  1.1060us     104ns  51.635us  cuDeviceGetAttribute
                    0.01%  12.388us         1  12.388us  12.388us  12.388us  cuDeviceGetName
                    0.00%  9.9530us         1  9.9530us  9.9530us  9.9530us  cudaDeviceSynchronize
                    0.00%  5.6130us         1  5.6130us  5.6130us  5.6130us  cuDeviceGetPCIBusId
                    0.00%  1.4870us         3     495ns     133ns  1.1100us  cuDevic

Above output shows the single `add` call. To see which GPU you have (e.g. Tesla T4) run:


In [7]:
%%shell

nvidia-smi

Sat Aug 30 16:16:04 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Let's make it faster with parallelism.

## Picking up the Threads

Execution config `<<<blocks, threadsPerBlock>>>`. Start by increasing threads per block: `<<<1,256>>>` (multiples of 32 are typical).


```cpp
add<<<1, 256>>>(N, x, y);
```

To partition work, use built-ins: `threadIdx.x` (thread id in block), `blockDim.x` (threads per block). Stride loop lets each thread handle a slice.


```cpp
__global__
void add(int n, float *x, float *y)
{
  int index = threadIdx.x;
  int stride = blockDim.x;
  for (int i = index; i < n; i += stride)
      y[i] = x[i] + y[i];
}
```

Setting `index=0`, `stride=1` matches original. Save as `add_block.cu`, compile, profile.


In [8]:
%%writefile add_block.cu

#include <iostream>
#include <math.h>

// Kernel function to add the elements of two arrays
__global__
void add(int n, float *x, float *y)
{
  int index = threadIdx.x;
  int stride = blockDim.x;
  for (int i = index; i < n; i += stride)
      y[i] = x[i] + y[i];
}

int main(void)
{
  int N = 1<<20;
  float *x, *y;

  // Allocate Unified Memory – accessible from CPU or GPU
  cudaMallocManaged(&x, N*sizeof(float));
  cudaMallocManaged(&y, N*sizeof(float));

  // initialize x and y arrays on the host
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Run kernel on 1M elements on the GPU
  add<<<1, 256>>>(N, x, y);

  // Wait for GPU to finish before accessing on host
  cudaDeviceSynchronize();

  // Check for errors (all values should be 3.0f)
  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = fmax(maxError, fabs(y[i]-3.0f));
  std::cout << "Max error: " << maxError << std::endl;

  // Free memory
  cudaFree(x);
  cudaFree(y);

  return 0;
}

Writing add_block.cu


In [9]:
%%shell

nvcc add_block.cu -o add_block
nvprof ./add_block

==1337== NVPROF is profiling process 1337, command: ./add_block
Max error: 1
==1337== Profiling application: ./add_block
==1337== Profiling result:
No kernels were profiled.
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
      API calls:   94.81%  227.23ms         2  113.62ms  62.254us  227.17ms  cudaMallocManaged
                    4.90%  11.755ms         1  11.755ms  11.755ms  11.755ms  cudaLaunchKernel
                    0.19%  459.23us         2  229.61us  219.73us  239.50us  cudaFree
                    0.07%  175.84us       114  1.5420us     143ns  70.650us  cuDeviceGetAttribute
                    0.01%  13.778us         1  13.778us  13.778us  13.778us  cuDeviceGetName
                    0.01%  13.019us         1  13.019us  13.019us  13.019us  cudaDeviceSynchronize
                    0.00%  7.5550us         1  7.5550us  7.5550us  7.5550us  cuDeviceGetPCIBusId
                    0.00%  2.4270us         3     809ns     188ns  1.9190us  cuDev

Expect ~256x less kernel time vs 1 thread (hardware + overheads may vary). Let's scale further with multiple blocks.


## Out of the Blocks

GPUs have many SMs running multiple blocks concurrently. Use multiple blocks to use more SMs. Compute blocks: `numBlocks = (N + blockSize - 1)/blockSize`.


```cpp
int blockSize = 256;
int numBlocks = (N + blockSize - 1) / blockSize;
add<<<numBlocks, blockSize>>>(N, x, y);
```

<img src="https://developer-blogs.nvidia.com/wp-content/uploads/2017/01/cuda_indexing.png" width="800">

Index formula: `globalIndex = blockIdx.x * blockDim.x + threadIdx.x`. Use loop stride = total threads: `blockDim.x * gridDim.x` (grid-stride loop) so any grid size covers N.


```cpp
__global__
void add(int n, float *x, float *y)
{
  int index = blockIdx.x * blockDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;
  for (int i = index; i < n; i += stride)
    y[i] = x[i] + y[i];
}
```

Stride = total threads (`blockDim.x * gridDim.x`). This grid-stride loop pattern scales automatically. Save as `add_grid.cu`, compile, profile.


In [10]:
%%writefile add_grid.cu

#include <iostream>
#include <math.h>

// Kernel function to add the elements of two arrays
__global__
void add(int n, float *x, float *y)
{
  int index = blockIdx.x * blockDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;
  for (int i = index; i < n; i += stride)
    y[i] = x[i] + y[i];
}

int main(void)
{
  int N = 1<<20;
  float *x, *y;

  // Allocate Unified Memory – accessible from CPU or GPU
  cudaMallocManaged(&x, N*sizeof(float));
  cudaMallocManaged(&y, N*sizeof(float));

  // initialize x and y arrays on the host
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Run kernel on 1M elements on the GPU
  int blockSize = 256;
  int numBlocks = (N + blockSize - 1) / blockSize;
  add<<<numBlocks, blockSize>>>(N, x, y);

  // Wait for GPU to finish before accessing on host
  cudaDeviceSynchronize();

  // Check for errors (all values should be 3.0f)
  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = fmax(maxError, fabs(y[i]-3.0f));
  std::cout << "Max error: " << maxError << std::endl;

  // Free memory
  cudaFree(x);
  cudaFree(y);

  return 0;
}

Writing add_grid.cu


In [11]:
%%shell

nvcc add_grid.cu -o add_grid
nvprof ./add_grid

==1443== NVPROF is profiling process 1443, command: ./add_grid
Max error: 1
==1443== Profiling application: ./add_grid
==1443== Profiling result:
No kernels were profiled.
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
      API calls:   95.98%  205.41ms         2  102.70ms  53.308us  205.35ms  cudaMallocManaged
                    3.77%  8.0760ms         1  8.0760ms  8.0760ms  8.0760ms  cudaLaunchKernel
                    0.16%  351.93us         2  175.96us  131.63us  220.30us  cudaFree
                    0.07%  147.59us       114  1.2940us     112ns  68.757us  cuDeviceGetAttribute
                    0.01%  12.679us         1  12.679us  12.679us  12.679us  cuDeviceGetName
                    0.01%  11.543us         1  11.543us  11.543us  11.543us  cudaDeviceSynchronize
                    0.00%  5.6790us         1  5.6790us  5.6790us  5.6790us  cuDeviceGetPCIBusId
                    0.00%  1.4370us         3     479ns     120ns  1.0490us  cuDevic

Multi-block launch yields another jump (results vary by GPU). If final speedups differ from blog, see Exercise 4.


## Exercises

Try these:
1. Browse [CUDA docs](https://docs.nvidia.com/cuda/index.html): install guides, Programming Guide, Best Practices, arch tuning guides.
2. Add `printf` in the kernel for `threadIdx.x`, `blockIdx.x`. Observe ordering (not guaranteed sequential).
3. Print `threadIdx.y/z`, `blockIdx.y`. Explore 2D/3D launch shapes (set in `<<<grid, block>>>`).
4. On a Pascal GPU compare `add_grid.cu` performance. Read about [Pascal UMA + page migration](https://developer.nvidia.com/blog/beyond-gpu-memory-limits-unified-memory-pascal/) and see [Unified Memory for CUDA Beginners](https://developer.nvidia.com/blog/unified-memory-cuda-beginners/).


## Where to From Here

Next steps:
- C/C++ course: [_Fundamentals of Accelerated Computing with CUDA C/C++_](https://courses.nvidia.com/courses/course-v1:DLI+C-AC-01+V1/about)
- Python course: [_Fundamentals of Accelerated Computing with CUDA Python_](https://courses.nvidia.com/courses/course-v1:DLI+C-AC-02+V1/about)
- More: DLI [_Accelerated Computing_ catalog](https://www.nvidia.com/en-us/training/online/)


## Quick CUDA Recap (What I Learned Here)
**Problem we solved:** Adding two large float arrays efficiently. We started with a plain CPU loop and progressively parallelized it on the GPU.

**Steps you walked through:**
1. CPU baseline: simple for-loop add.
2. Turned the function into a CUDA kernel (`__global__`).
3. Used Unified Memory (`cudaMallocManaged`) so one pointer works on host & device.
4. Launched a kernel with 1 thread (correct but slow) `add<<<1,1>>>`.
5. Introduced many threads in one block (`threadIdx.x` + stride) to split the work.
6. Scaled to many blocks so multiple SMs run in parallel (`blockIdx.x * blockDim.x + threadIdx.x`).
7. Added a grid‑stride loop so any grid size still covers all N elements.
8. Profiled runs (`nvprof`) and inspected GPU info (`nvidia-smi`).

**Core concepts gained:**
- Kernel launch syntax `<<<grid, block>>>`.
- Thread & block indexing math.
- Grid‑stride loop pattern for scalable kernels.
- Unified Memory for fast prototyping (simpler than manual `cudaMemcpy`).
- Need for `cudaDeviceSynchronize()` before reading results.
- Validating correctness (max error check) after GPU execution.

**Mental template you can now reuse:**
Prepare data → allocate (Unified or device) → compute global index → loop with stride → write results → synchronize → validate.

**Next easy extensions:** Try a different element-wise op (scale, bias, clamp), then a reduction (sum) using shared memory, then fuse multiple operations into one kernel.

**One sentence:** You learned how to map a simple array operation from a single CPU loop to thousands of GPU threads using CUDA kernels, indexing, and grid‑stride loops while ensuring correctness and scalability.